<a href="https://colab.research.google.com/github/brendanpshea/ibm-data-science/blob/main/jupyter_labs_eda_sql_coursera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Brendan's Notes
This is based on an IBM lab, but with substantial modifications. In particular, I'm using SQLite and Pandas, as opposed to DB2. Things I learned:

1. How to create SQLite connections in SQLalchemy in a number of different ways.
2. How to convert SQL and Pandas back and forth.
3. How to wrangle with dates in SQLite
4. Why it would have been better to do this all using Postgres (seriously!).

## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones.

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage.

Therefore if we can determine if the first stage will land, we can determine the cost of a launch.

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


### Download the datasets

This assignment requires you to load the spacex dataset.

In many cases the dataset to be analyzed is available as a .CSV (comma separated values) file, perhaps on the internet. Click on the link below to download and save the dataset (.CSV file):

<a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2021-01-01" target="_blank">Spacex DataSet</a>


In [1]:
!pip install sqlalchemy
!pip install ipython-sql
!pip install wget

### Connect to the database

Let us first load the SQL extension and establish a connection with the database


In [2]:
%load_ext sql

In [3]:
!wget "https://github.com/brendanpshea/ibm-data-science/raw/main/spacex_sqlite.db"

--2022-02-08 21:08:12--  https://github.com/brendanpshea/ibm-data-science/raw/main/spacex_sqlite.db
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/brendanpshea/ibm-data-science/main/spacex_sqlite.db [following]
--2022-02-08 21:08:13--  https://raw.githubusercontent.com/brendanpshea/ibm-data-science/main/spacex_sqlite.db
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36864 (36K) [application/octet-stream]
Saving to: ‘spacex_sqlite.db’

spacex_sqlite.db    100%[===================>]  36.00K  --.-KB/s    in 0.003s  

2022-02-08 21:08:13 (11.1 MB/s) - ‘spacex_sqlite.db’ saved [36864/36864]



In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
from sqlalchemy import create_engine

# db_uri = "sqlite:////content/drive/MyDrive/ibm_sqlite_1.db"
db_uri = "sqlite:////content/spacex_sqlite.db"
engine = create_engine(db_uri)

# %sql sqlite:///content/drive/MyDrive/ibm_sqlite_1.db
%sql sqlite:////content/spacex_sqlite.db

'Connected: @/content/spacex_sqlite.db'

In [ ]:
engine("select *")

## Tasks

Now write and execute SQL queries to solve the assignment tasks.

### Task 1

##### Display the names of the unique launch sites  in the space mission


In [6]:
%sql select distinct launch_site from spacex;

 * sqlite:////content/spacex_sqlite.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


In [7]:
%%sql 
select * from spacex limit 5;

 * sqlite:////content/spacex_sqlite.db
Done.


Date,Time(UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
04-06-2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
08-12-2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
22-05-2012,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
08-10-2012,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
01-03-2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


The "pragma" gives us the table info. Neat!

In [8]:
%sql pragma table_info(spacex)

 * sqlite:////content/spacex_sqlite.db
Done.


cid,name,type,notnull,dflt_value,pk
0,Date,TEXT,0,None,0
1,Time(UTC),TEXT,0,None,0
2,Booster_Version,TEXT,0,None,0
3,Launch_Site,TEXT,0,None,0
4,Payload,TEXT,0,None,0
5,PAYLOAD_MASS__KG_,INTEGER,0,None,0
6,Orbit,TEXT,0,None,0
7,Customer,TEXT,0,None,0
8,Mission_Outcome,TEXT,0,None,0
9,Landing_Outcome,TEXT,0,None,0


### Task 2

##### Display 5 records where launch sites begin with the string 'CCA'


In [9]:
%sql select * from spacex where launch_site like 'CCA%' limit 5;

 * sqlite:////content/spacex_sqlite.db
Done.


Date,Time(UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
04-06-2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
08-12-2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
22-05-2012,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
08-10-2012,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
01-03-2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### Task 3

##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [10]:
%sql select sum(PAYLOAD_MASS__KG_) from spacex where customer = 'NASA (CRS)'

 * sqlite:////content/spacex_sqlite.db
Done.


sum(PAYLOAD_MASS__KG_)
45596


### Task 4

##### Display average payload mass carried by booster version F9 v1.1


In [11]:
%sql select avg(PAYLOAD_MASS__KG_) from spacex where Booster_Version = 'F9 v1.1'

 * sqlite:////content/spacex_sqlite.db
Done.


avg(PAYLOAD_MASS__KG_)
2928.4


### Task 5

##### List the date when the first successful landing outcome in ground pad was acheived.

*Hint:Use min function*


In [12]:
%sql select distinct landing_outcome from spacex

 * sqlite:////content/spacex_sqlite.db
Done.


Landing_Outcome
Failure (parachute)
No attempt
Uncontrolled (ocean)
Controlled (ocean)
Failure (drone ship)
Precluded (drone ship)
Success (ground pad)
Success (drone ship)
Success
Failure


This cell below was a fair amount of work, but it shows how to load a sql result set to pandas and then parse datetimes using pandas!

In [13]:
import pandas as pd

my_dates = %sql select date from spacex where landing_outcome = 'Success (ground pad)'

date_df = pd.DataFrame(my_dates)
date_df.columns = ['Dates']
pd.to_datetime(date_df['Dates']).min()

 * sqlite:////content/spacex_sqlite.db
Done.


Timestamp('2015-12-22 00:00:00')

### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [14]:
%%sql 
select booster_version from spacex 
  where landing_outcome = 'Success (drone ship)' and
  PAYLOAD_MASS__KG_ between 4000 and 6000

 * sqlite:////content/spacex_sqlite.db
Done.


Booster_Version
F9 FT B1022
F9 FT B1026
F9 FT B1021.2
F9 FT B1031.2


### Task 7

##### List the total number of successful and failure mission outcomes


In [15]:
%%sql

select count((select landing_outcome from spacex where landing_outcome like 'Success%')) as "Successes", 
 count((select landing_outcome from spacex where landing_outcome like 'Failure%')) as "Failures"
 from spacex

 * sqlite:////content/spacex_sqlite.db
Done.


Successes,Failures
101,101


### Task 8

##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [16]:
%%sql

select booster_version from spacex where PAYLOAD_MASS__KG_ = 
  (select max(PAYLOAD_MASS__KG_) from spacex)

 * sqlite:////content/spacex_sqlite.db
Done.


Booster_Version
F9 B5 B1048.4
F9 B5 B1049.4
F9 B5 B1051.3
F9 B5 B1056.4
F9 B5 B1048.5
F9 B5 B1051.4
F9 B5 B1049.5
F9 B5 B1060.2
F9 B5 B1058.3
F9 B5 B1051.6


### Task 9

##### List the failed landing_outcomes in drone ship, their booster versions, and launch site names for in year 2015


In [17]:
%sql select Booster_Version, launch_site from spacex limit 5

 * sqlite:////content/spacex_sqlite.db
Done.


Booster_Version,Launch_Site
F9 v1.0 B0003,CCAFS LC-40
F9 v1.0 B0004,CCAFS LC-40
F9 v1.0 B0005,CCAFS LC-40
F9 v1.0 B0006,CCAFS LC-40
F9 v1.0 B0007,CCAFS LC-40


In [18]:
%%sql

select Booster_Version, launch_site from spacex
  where landing_outcome = 'Failure (drone ship)' and
  date like '%2015'

 * sqlite:////content/spacex_sqlite.db
Done.


Booster_Version,Launch_Site
F9 v1.1 B1012,CCAFS LC-40
F9 v1.1 B1015,CCAFS LC-40


### Task 10

##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order

Brendan's Note: I did some neat stuff here. I read a table into pandas, converted strings into datetime for comparison, then wrote it back out the SQL engine.


In [19]:
results = %sql select * from spacex

 * sqlite:////content/spacex_sqlite.db
Done.


In [24]:
df = pd.DataFrame(results)
df.head()

,0,1,2,3,4,5,6,7,8,9
0,04-06-2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,08-12-2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,22-05-2012,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,08-10-2012,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,01-03-2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


In [30]:
conn = engine.connect()

In [34]:
# begin by clearning out table if it already exists
%sql drop table if exists q10Table

query = "select * from spacex"
df2 = pd.read_sql(query, conn)
df2['Date'] = pd.to_datetime(df2['Date'])
q10_list = df2[(df2['Date'] > pd.to_datetime('2010-06-04')) & (df2['Date'] < pd.to_datetime('2017-03-20'))]
q10_list.to_sql('q10Table', engine)

 * sqlite:////content/spacex_sqlite.db
Done.


In [21]:
%sql select * from q10Table

 * sqlite:////content/spacex_sqlite.db
Done.


index,Date,Time(UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
1,2010-08-12 00:00:00.000000,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2012-05-22 00:00:00.000000,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,2012-08-10 00:00:00.000000,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,2013-01-03 00:00:00.000000,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt
5,2013-09-29 00:00:00.000000,16:00:00,F9 v1.1 B1003,VAFB SLC-4E,CASSIOPE,500,Polar LEO,MDA,Success,Uncontrolled (ocean)
6,2013-03-12 00:00:00.000000,22:41:00,F9 v1.1,CCAFS LC-40,SES-8,3170,GTO,SES,Success,No attempt
7,2014-06-01 00:00:00.000000,22:06:00,F9 v1.1,CCAFS LC-40,Thaicom 6,3325,GTO,Thaicom,Success,No attempt
8,2014-04-18 00:00:00.000000,19:25:00,F9 v1.1,CCAFS LC-40,SpaceX CRS-3,2296,LEO (ISS),NASA (CRS),Success,Controlled (ocean)
9,2014-07-14 00:00:00.000000,15:15:00,F9 v1.1,CCAFS LC-40,OG2 Mission 1 6 Orbcomm-OG2 satellites,1316,LEO,Orbcomm,Success,Controlled (ocean)
10,2014-05-08 00:00:00.000000,08:00:00,F9 v1.1,CCAFS LC-40,AsiaSat 8,4535,GTO,AsiaSat,Success,No attempt


In [22]:
%sql select landing_outcome, count(landing_outcome) from q10Table group by landing_outcome order by count(landing_outcome)

 * sqlite:////content/spacex_sqlite.db
Done.


Landing_Outcome,count(landing_outcome)
Failure (parachute),1
Precluded (drone ship),1
Uncontrolled (ocean),2
Controlled (ocean),3
Failure (drone ship),5
Success (drone ship),5
Success (ground pad),5
No attempt,10


## Author(s)

<h4> Lakshmi Holla </h4>


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
